In [54]:
# get WRDS data
import wrds
conn = wrds.Connection(wrds_username='deadtomb')

Loading library list...
Done


In [55]:
import pandas as pd

In [56]:
conn.create_pgpass_file()

In [57]:
conn.list_tables(library='crsp_a_ccm')

['ccm_lookup',
 'ccm_qvards',
 'ccmxpf_linktable',
 'ccmxpf_lnkhist',
 'ccmxpf_lnkrng',
 'ccmxpf_lnkused',
 'comphead',
 'comphist',
 'compmaster',
 'cst_hist',
 'sechead',
 'sechist']

In [58]:
conn.describe_table(library='crsp_a_ccm', table='ccmxpf_linktable')

Approximately 85574 rows in crsp_a_ccm.ccmxpf_linktable.


,name,nullable,type,comment
0,gvkey,True,VARCHAR(6),Standard and Poor's Identifier
1,linkprim,True,VARCHAR(1),Primary Link Marker
2,liid,True,VARCHAR(3),Security-level Identifer
3,linktype,True,VARCHAR(2),Link Type Code
4,lpermno,True,DOUBLE PRECISION,Historical CRSP PERMNO Link to COMPUSTAT Record
5,lpermco,True,DOUBLE PRECISION,Historical CRSP PERMCO Link to COMPUSTAT Record
6,usedflag,True,DOUBLE PRECISION,Flag marking whether link is used in building ...
7,linkdt,True,DATE,First Effective Date of Link
8,linkenddt,True,DATE,Last Effective Date of Link


In [59]:
link_table = conn.raw_sql("""
    SELECT gvkey, lpermno AS permno, linktype, linkprim, linkdt, linkenddt
    FROM crsp.ccmxpf_linktable
    WHERE linktype IN ('LU', 'LC') AND linkprim IN ('P', 'C')
""")

In [60]:
compustat_data = conn.raw_sql("""
    SELECT gvkey, fyear, datadate, at, ceq, csho, prcc_f, dlc, dltt, dvc, oibdp, sale, ppegt, xad, xrd, fyr
    FROM comp.funda
    WHERE indfmt = 'INDL' AND datafmt = 'STD' AND popsrc = 'D' AND consol = 'C' AND fyear BETWEEN 1950 AND 2021
""")